In [ ]:
import numpy as np
import pandas as pd
import json
import requests
import time
import csv
from sklearn.linear_model import LinearRegression

# **Construction du jeu de données avec notre analyse tirée de l'article:**
"*A Query Sampling Metho d for Estimating Local Cost Parameters in a Multidatabase System*"*

les auteurs utilisent une technique de sampling de requêtes (échantillonnage de requêtes) pour estimer les paramètres de coût des requêtes dans un système de bases de données multidatabase (MDBS)

ALors Pour adapter cette méthode à notre context nous procédons comme suit:


  **Etape 1:**   Classification des classes dans laquelle nous  retenons deux grande classes principales:

              1.   Requete Unaire
              2.   Requete de jointure


  **Étape 2 :** Échantillonnage des requêtes
  Pour chaque grande classe de requete nous créeons des échantillons de requetes.

    Par exemple pour la classe unary nous créons des groupes d'echantillons par type:

    1. G11 Requêtes simples avec agrégation (par exemple, COUNT, SUM).
    2. G12 : Requêtes avec GROUP BY.
    3. G13 : Requêtes avec filtrage (WHERE, HAVING) et post-traitement (ORDER BY, LIMIT, DISTINCT).


**Étape 3 :** Exécution des requêtes et collecte des informations sur chaque echantilons sur les sources de données que nous avons (API, etc) telles que: **La cardinalité, la selectivité et le coût observé en seconde.**


**Étape 4 :** Détermination des coefficients de régression

  Pour chaque classe de requêtes, nous utilisons une formule de coût. Par exemple :

  

    1.   la classe requete unaire:
      cout  = beta0 + beta1 * N + beta2 * S * N
      avec N--> nombre d'element dans l'operande et S--> la selectivité de la requete (nombre de tuple satisfaisant la condition de restriction)

    2.   la classe Jointure:

      cout = beta0 + beta1 * N + beta1*N2 + beta2 * S * N1 * N2



**Etape 5:** Pour implémenter ça nous utilisons des
outils pour la régression multiple(abordé dans l'article) .   
On utilise Python avec scikit-learn






# **Dans un premier temps nous nous  s'interessons qu'à la classe requete Unaire d'abord sur les API**

## **API Consommation annuelle d’électricité et gaz par EPCI et par secteur d’activité (jusqu'en 2021)**:

url = https://opendata.agenceore.fr/api/explore/v2.1/catalog/datasets/conso-elec-gaz-annuelle-par-secteur-dactivite-agregee-epci/records


In [ ]:
source ="API "
url_api1 ='https://opendata.agenceore.fr/api/explore/v2.1/catalog/datasets/conso-elec-gaz-annuelle-par-secteur-dactivite-agregee-epci/records'


queries_G11 =['?select=operateur,COUNT(code_epci)&group_by=operateur&limit=100',
              '?select=operateur,AVG(consototale)&group_by=operateur',
              '?select=libelle_departement,SUM(consototale)&group_by=libelle_departement',
              '?select=libelle_region,COUNT(DISTINCT code_epci)&group_by=libelle_region',
              '?select=count(distinct libelle_region)',
              '?select=COUNT(distinct code_epci)',
              '?select=SUM(consototale)',
              '?select=MAX(consototale)',
              '?select=AVG(consototale)'
              ]


queries_G12 =['?select=libelle_region,SUM(consototale)&group_by=libelle_region&limit=-1',
              '?select=operateur,COUNT(code_epci)&group_by=operateur&limit=100',
              '?select=operateur,AVG(consototale)&group_by=operateur&limit=-1',
              '?select=libelle_region,AVG(consototale)&group_by=libelle_region&limit=-1',
              '?select=libelle_region,COUNT(code_epci)&group_by=libelle_region&limit=-1',
              '?select=libelle_region,MAX(consototale)&group_by=libelle_region&limit=-1',
              '?select=libelle_region,MIN(consototale)&group_by=libelle_region&limit=-1',
              '?select=libelle_region,SUM(consototale)&group_by=libelle_region&having=SUM(consototale)>5000000&limit=100',
              '?select=libelle_region,SUM(consototale)&group_by=libelle_region&order_by=SUM(consototale) ASC&limit=50',
              '?select=libelle_region,SUM(consototale)&group_by=libelle_region&order_by=SUM(consototale) DESC&limit=5',
              '?select=libelle_epci,COUNT(*)&group_by=libelle_epci&order_by=count(*)desc&limit=20',
              '?select=libelle_epci,COUNT(*)&group_by=libelle_epci&order_by=COUNT(*) DESC',
              '?select=libelle_departement,AVG(consototale)&group_by=libelle_departement&order_by=AVG(consototale) DESC&limit=10',
              '?select=operateur,COUNT(code_epci)&group_by=operateur&having=COUNT(code_epci)>50&limit=-1',
              '?select=libelle_region,MIN(consototale)&group_by=libelle_region&order_by=MIN(consototale) ASC&limit=5',
              '?select=libelle_epci,SUM(consototale)&group_by=libelle_epci&limit=50',
              '?select=libelle_region,COUNT(DISTINCT libelle_departement)&group_by=libelle_region&limit=-1',
              '?select=libelle_departement,SUM(consototale)&group_by=libelle_departement&limit=-1',
              '?select=libelle_region,COUNT(DISTINCT code_epci)&group_by=libelle_region&limit=-1',
              '?select=libelle_region,COUNT(*)&group_by=libelle_region&limit=-1'
              ]

query_G13 = ['?where=consototale>500000 and consototale<1000000&limit=-1',
             "?where=libelle_region!='Île-de-France'&limit=-1",
             "?where=libelle_region='Nouvelle-Aquitaine'&select=AVG(consototale)&limit=20",
             "?where=libelle_region='Occitanie'&limit=-1",
             "?where=libelle_region='Occitanie'&limit=50",
             '?where=consototale>500000 and consototale<1000000&limit=100',
             "?where=libelle_epci LIKE 'Grand%'&limit=-1",
             '?where=consototale>10000000&limit=-1',
             "?where=consototale<100000 AND libelle_region='Bretagne'&limit=100",
             "?where=libelle_region LIKE '%Nouvelle%'&limit=50",
             '?where=consototale>2000000 and libelle_region="Grand Est"&limit=-1',
            '?where=libelle_departement LIKE "P%"&limit=100',
            '?where=consototale<50000&limit=50',
            '?where=operateur="ENEDIS" and consototale>100000&limit=-1',
            '?where=libelle_region IN ("Bretagne", "Occitanie")&limit=-1'
             ]

## **Second API: Consommation EPCI annuelle 2012-2023**

url = https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/conso-epci-annuelle/exports/json"


In [ ]:
url_api2 = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/conso-epci-annuelle/exports/json"

queries_api2_G11 =['?select=operateurs,COUNT(code_epci)&group_by=operateurs&limit=100',
              '?select=operateurs,AVG(consototale)&group_by=operateurs',
              '?select=libelle_departements,SUM(consototale)&group_by=libelle_departements',
              '?select=libelle_regions,COUNT(DISTINCT code_epci)&group_by=libelle_regions',
              '?select=count(distinct libelle_regions)',
              '?select=COUNT(distinct code_epci)',
              '?select=SUM(consototale)',
              '?select=MAX(consototale)',
              '?select=AVG(consototale)'
              ]

queries_api2_G12 =['?select=libelle_regions,SUM(consototale)&group_by=libelle_regions',
              '?select=libelle_regions,AVG(consototale)&group_by=libelle_regions',
              '?select=libelle_regions,COUNT(code_epci)&group_by=libelle_regions',
              '?select=libelle_regions,MAX(consototale)&group_by=libelle_regions',
              '?select=libelle_regions,MIN(consototale)&group_by=libelle_regions',
              '?select=libelle_regions,SUM(consototale)&group_by=libelle_regions&having=SUM(consototale)>5000000&limit=100',
              '?select=libelle_regions,SUM(consototale)&group_by=libelle_regions&order_by=SUM(consototale) ASC&limit=50',
              '?select=libelle_regions,SUM(consototale)&group_by=libelle_regions&order_by=SUM(consototale) DESC&limit=5',
              '?select=libelle_epci,COUNT(*)&group_by=libelle_epci&order_by=count(*)desc&limit=20',
              '?select=libelle_epci,COUNT(*)&group_by=libelle_epci&order_by=COUNT(*) desc',
              '?select=libelle_regions,COUNT(*)&group_by=libelle_regions',
              '?select=libelle_departements,AVG(consototale)&group_by=libelle_departements&order_by=AVG(consototale) DESC&limit=10',
              '?select=operateurs,COUNT(code_epci)&group_by=operateurs&having=COUNT(code_epci)>50&limit=-1',
              '?select=libelle_regions,MIN(consototale)&group_by=libelle_regions&order_by=MIN(consototale) ASC&limit=5',
              '?select=libelle_epci,SUM(consototale)&group_by=libelle_epci&limit=50',
              '?select=libelle_regions,COUNT(DISTINCT libelle_departements)&group_by=libelle_regions&limit=-1'
              ]

query_api2_G13 = ['?where=consototale>500000 and consototale<1000000',
             "?where=libelle_regions!='Île-de-France'",
             "?where=libelle_regions='Nouvelle-Aquitaine'&select=AVG(consototale)&limit=20",
             "?where=libelle_regions='Occitanie'",
             "?where=libelle_regions='Occitanie'&limit=50",
             '?where=consototale>500000 and consototale<1000000',
             "?where=libelle_epci LIKE 'Grand%'",
             '?where=consototale>10000000',
             "?where=consototale<100000 AND libelle_regions='Bretagne'",
             "?where=libelle_regions LIKE '%Nouvelle%'&limit=50"
             ]

## Implémentation d'un modele de regression multiple

Dans l'article "A Query Sampling Method for Estimating Local Cost Parameters in a Multidatabase System", les auteurs ont testé leur modèle en suivant une méthodologie expérimentale basée sur l'échantillonnage de requêtes et la régression multiple. Voici les étapes clés qu'ils ont suivies :

In [ ]:
# Organisation des requêtes dans une liste de sous-groupes
queries_api1 = [queries_G11, queries_G12, query_G13]
queries_api2 = [queries_api2_G11, queries_api2_G12, query_api2_G13]
sub_classes = ["G11", "G12", "G13"]  # Nom des sous-groupes correspondants

In [ ]:
# Fonction pour obtenir la cardinalité totale (N)
def get_total_records(url):
    N_total_query = "?select=COUNT(*)"
    response = requests.get(url + N_total_query)
    if response.ok:
        N_total = response.json()["results"][0]["COUNT(*)"]
        return N_total
    else:
        print(f"Erreur lors de la récupération de N : {response.status_code}")
        return None

In [ ]:
# Fonction pour classer les requêtes (Unary ou Jointure)
def classify_query(query):
    if "join" in query:
        return "Jointure"
    return "Unary"

In [ ]:
# Fonction pour exécuter les requêtes et collecter les données avec sous-groupes
def build_data(csv_name, queries_list, source_name, url_api, N_total):
    data = []
    for sub_group_idx, queries in enumerate(queries_list):
        sub_class = sub_classes[sub_group_idx]  # Associe G11, G12 ou G13
        for query in queries:
            full_url = url_api + query
            start_time = time.time()
            response = requests.get(full_url)
            end_time = time.time()

            if response.ok:
                json_data = response.json()
                if isinstance(json_data, list):
                    results = json_data  # If it's a list, assign it directly to results
                else:
                    results = json_data.get("results", [])  # If it's a dictionary, use get()

                N_selected = len(results)  # Nombre de tuples retournés
                S = N_selected / N_total if N_total else None  # Sélectivité
                cost = round(end_time - start_time, 4)  # Coût en secondes

                data.append([source_name, query, classify_query(query), sub_class, N_total, N_selected, S, cost])
                print(f"Requête traitée ({sub_class}) : {query}")
            else:
                print(f"Erreur pour {query} ({sub_class}) - Code {response.status_code}")

    # Création du DataFrame avec la nouvelle colonne "sub_class"
    df = pd.DataFrame(data, columns=["Source", "Query", "QueryClass", "sub_class", "N", "Nb tuples selected", "S", "Cost (sec)"])
    df.to_csv(csv_name, index=False, encoding="utf-8-sig")
    print(f"Résultats enregistrés dans {csv_name}")
    return df



# **Execution pour la contruction du jeu de données**

In [ ]:
# Exécution
cardinality_api1 = get_total_records(url_api1)
if cardinality_api1:
    print(f"Cardinalité totale (N) pour API 1 : {cardinality_api1}")
    df_result = build_data("Api1_query_samples_with_subclass.csv", queries_api1, "API1", url_api1, cardinality_api1)
else:
    print("Impossible de continuer sans cardinalité.")

Cardinalité totale (N) pour API 1 : 38035
Requête traitée (G11) : ?select=operateur,COUNT(code_epci)&group_by=operateur&limit=100
Requête traitée (G11) : ?select=operateur,AVG(consototale)&group_by=operateur
Requête traitée (G11) : ?select=libelle_departement,SUM(consototale)&group_by=libelle_departement
Requête traitée (G11) : ?select=libelle_region,COUNT(DISTINCT code_epci)&group_by=libelle_region
Requête traitée (G11) : ?select=count(distinct libelle_region)
Requête traitée (G11) : ?select=COUNT(distinct code_epci)
Requête traitée (G11) : ?select=SUM(consototale)
Requête traitée (G11) : ?select=MAX(consototale)
Requête traitée (G11) : ?select=AVG(consototale)
Requête traitée (G12) : ?select=libelle_region,SUM(consototale)&group_by=libelle_region&limit=-1
Requête traitée (G12) : ?select=operateur,COUNT(code_epci)&group_by=operateur&limit=100
Requête traitée (G12) : ?select=operateur,AVG(consototale)&group_by=operateur&limit=-1
Requête traitée (G12) : ?select=libelle_region,AVG(consot

In [ ]:
df_result = build_data("Api2_query_samples_with_subclass.csv", queries_api2, "API2", url_api2, 15026)

Requête traitée (G11) : ?select=operateurs,COUNT(code_epci)&group_by=operateurs&limit=100
Requête traitée (G11) : ?select=operateurs,AVG(consototale)&group_by=operateurs
Requête traitée (G11) : ?select=libelle_departements,SUM(consototale)&group_by=libelle_departements
Requête traitée (G11) : ?select=libelle_regions,COUNT(DISTINCT code_epci)&group_by=libelle_regions
Requête traitée (G11) : ?select=count(distinct libelle_regions)
Requête traitée (G11) : ?select=COUNT(distinct code_epci)
Requête traitée (G11) : ?select=SUM(consototale)
Requête traitée (G11) : ?select=MAX(consototale)
Requête traitée (G11) : ?select=AVG(consototale)
Requête traitée (G12) : ?select=libelle_regions,SUM(consototale)&group_by=libelle_regions
Requête traitée (G12) : ?select=libelle_regions,AVG(consototale)&group_by=libelle_regions
Requête traitée (G12) : ?select=libelle_regions,COUNT(code_epci)&group_by=libelle_regions
Requête traitée (G12) : ?select=libelle_regions,MAX(consototale)&group_by=libelle_regions
Re

## Implémentation d'un modele de regression multiple

In [ ]:
#Chargement des fichiers csv
data_api1 = pd.read_csv("Api1_query_samples_with_subclass.csv")
data_api2 = pd.read_csv("Api2_query_samples_with_subclass.csv")

In [ ]:
#Fusion des fichiers csv
global_data = pd.concat([data_api1, data_api2],ignore_index=True)
global_data.head()

,Source,Query,QueryClass,sub_class,N,Nb tuples selected,S,Cost (sec)
0,API1,"?select=operateur,COUNT(code_epci)&group_by=op...",Unary,G11,38035,100,0.002629,0.2173
1,API1,"?select=operateur,AVG(consototale)&group_by=op...",Unary,G11,38035,181,0.004759,0.1807
2,API1,"?select=libelle_departement,SUM(consototale)&g...",Unary,G11,38035,101,0.002655,0.1571
3,API1,"?select=libelle_region,COUNT(DISTINCT code_epc...",Unary,G11,38035,18,0.000473,0.1931
4,API1,?select=count(distinct libelle_region),Unary,G11,38035,10,0.000263,0.1502


In [ ]:
# Liste pour stocker les coefficients
coefficients_list = []

# Boucle sur chaque source unique
for source in global_data['Source'].unique():
    print(f"\nEntraînement sur les données de la source : {source}")

    # Filtrage des données par source
    df_source = global_data[global_data["Source"] == source]

    # Boucle sur chaque sous-groupe unique dans cette source
    for sub_class in df_source["sub_class"].unique():
        print(f"  Sous-groupe : {sub_class}")

        # Filtrage des données par sous-groupe
        df_subgroup = df_source[df_source["sub_class"] == sub_class]

        # Préparations des variables indépendantes (X) et dépendantes (Y)
        X = df_subgroup[["N", "S"]].copy()
        X.loc[:, "S"] = X["S"] * X["N"]  # Calcul de S * N
        print(f" Données X pour {sub_class} :")
        print(X.head())

        Y_observed = df_subgroup["Cost (sec)"]

        # Entraînement du modèle de régression linéaire
        model = LinearRegression()
        model.fit(X, Y_observed)

        # Calculer le score R²
        r2_score = model.score(X, Y_observed)
        print(f"    R² pour {sub_class} : {r2_score}")

        # Récupérer les coefficients et l'intercept
        coefficients = model.coef_
        intercept = model.intercept_

        # Ajouter les résultats à la liste
        coefficients_list.append({
            "source": source,
            "sub_class": sub_class,
            "beta0": intercept,
            "beta1": coefficients[0],
            "beta2": coefficients[1],
            "R2": r2_score
        })

        print(f"    Coefficients pour {sub_class} :")
        print(f"      beta0 (intercept) : {intercept}")
        print(f"      beta1 (N) : {coefficients[0]}")
        print(f"      beta2 (S*N) : {coefficients[1]}")
        print("    ---")

# Afficher tous les coefficients calculés
print("\nListe complète des coefficients :")
for coef in coefficients_list:
    print(coef)

# Sauvegarder les coefficients dans un fichier CSV
coefficients_df = pd.DataFrame(coefficients_list)
coefficients_df.to_csv("coefficients_by_source_and_subclass.csv", index=False, encoding="utf-8-sig")
print("\nCoefficients sauvegardés dans 'coefficients_by_source_and_subclass.csv'")


Entraînement sur les données de la source : API1
  Sous-groupe : G11
 Données X pour G11 :
       N      S
0  38035  100.0
1  38035  181.0
2  38035  101.0
3  38035   18.0
4  38035   10.0
    R² pour G11 : 0.2572263547698689
    Coefficients pour G11 :
      beta0 (intercept) : 0.1560069736552385
      beta1 (N) : 0.0
      beta2 (S*N) : 0.00020563830467301721
    ---
  Sous-groupe : G12
 Données X pour G12 :
        N      S
9   38035   18.0
10  38035  100.0
11  38035  181.0
12  38035   18.0
13  38035   18.0
    R² pour G12 : 0.00022310222847710115
    Coefficients pour G12 :
      beta0 (intercept) : 0.21029297689463178
      beta1 (N) : 0.0
      beta2 (S*N) : -8.657496592826148e-06
    ---
  Sous-groupe : G13
 Données X pour G13 :
        N      S
29  38035  100.0
30  38035  100.0
31  38035   20.0
32  38035  100.0
33  38035   50.0
    R² pour G13 : 6.36829070976841e-05
    Coefficients pour G13 :
      beta0 (intercept) : 0.1795079302674453
      beta1 (N) : 0.0
      beta2 (S*N) :